In [11]:
!pip install bitsandbytes
!pip install accelerate
!pip install transformers
!pip install langchain
!pip install sentence-transformers
!pip install pypdf
!pip install chromadb
!pip install accelerate

In [13]:
!pip install accelerate

In [6]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

In [15]:
# For the model quantization to be able to run in a free Google Colab notebook
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
# Version of the Mistral model used
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# Model and tokenizer
model_4bit = AutoModelForCausalLM.from_pretrained( model_id,quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained(model_id)

# So input sequence in model, embedded in integers, to tokens. tokenizer splits up sentence
# Model is no more than dictionary of tokens to integers. then inverse, integers get redone into tokens

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`